In [ ]:
# LImpar Arquivos
import os
import shutil

def clean_directory(exclude_extensions=['.ipynb']):
    # Obtém o diretório de trabalho atual
    current_directory = os.getcwd()

    try:
        # Lista todos os arquivos no diretório
        all_files = os.listdir(current_directory)

        # Filtra os arquivos a serem excluídos
        files_to_exclude = [f for f in all_files if not any(f.lower().endswith(ext) for ext in exclude_extensions)]

        # Exclui os arquivos
        for file_to_exclude in files_to_exclude:
            file_path = os.path.join(current_directory, file_to_exclude)
            if os.path.isfile(file_path):
                os.remove(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)

        print("Limpeza concluída. Arquivos excluídos com sucesso.")

    except Exception as e:
        print(f"Erro durante a limpeza do diretório: {e}")

# Executa a função para limpar o diretório (excluindo todos os arquivos, exceto .ipynb)
clean_directory(exclude_extensions=['.ipynb'])

In [ ]:
# Baixar imagem 
import os
import requests
from PIL import Image
from io import BytesIO
from urllib.parse import urlparse

def download_and_convert_to_jpg(url, save_directory="."):
    try:
        # Faz o download da imagem
        response = requests.get(url)
        response.raise_for_status()

        # Extrai o nome do arquivo da URL
        parsed_url = urlparse(url)
        filename = os.path.basename(parsed_url.path)

        # Monta o caminho completo para salvar a imagem
        save_path = os.path.join(save_directory, filename)

        # Lê a imagem em um objeto BytesIO
        image_data = BytesIO(response.content)

        # Abre a imagem usando a PIL
        original_image = Image.open(image_data)

        # Verifica a extensão da imagem
        _, extension = os.path.splitext(save_path)
        if extension.lower() != '.jpg':
            # Se a extensão não for .jpg, converte para JPEG
            save_path = os.path.splitext(save_path)[0] + '.jpg'
            original_image = original_image.convert('RGB')

        # Salva a imagem
        original_image.save(save_path)
        print(f"Imagem salva em {save_path}")

    except Exception as e:
        print(f"Erro durante o processamento da imagem: {e}")

# URL da imagem
image_url = "https://huggingface.co/ramsformers/shoes-brand/resolve/main/images/nike_shoes.jpg"

# Chama a função para baixar e converter a imagem
download_and_convert_to_jpg(image_url)

In [ ]:
# Redimencionar imagens
from PIL import Image
import os

def reduce_resolution(image, reduction_percentage):
    width, height = image.size
    new_width = max(int(width * (1 - reduction_percentage / 100)), 1)
    new_height = max(int(height * (1 - reduction_percentage / 100)), 1)
    return image.resize((new_width, new_height))

def main(script_directory):
    # Lista todos os arquivos .jpg no mesmo diretório do script
    jpg_files = [f for f in os.listdir(script_directory) if f.lower().endswith('.jpg')]

    if not jpg_files:
        print("No .jpg files found in the script directory.")
        return

    min_resolution_percentage = 2
    max_resolution_percentage = 100
    # Espaço entre porcentagens
    step_percentage = 2

    for percentage in range(max_resolution_percentage, min_resolution_percentage - 1, -step_percentage):
        for image_filename in jpg_files:
            image_path = os.path.join(script_directory, image_filename)

            try:
                original_image = Image.open(image_path)

                resized_image = reduce_resolution(original_image, percentage)
                new_width, new_height = resized_image.size
                new_filename = f"resized_{percentage}_{new_width}x{new_height}_{image_filename}"
                resized_image.save(os.path.join(script_directory, new_filename))
                print(f"Saved {new_filename}")

            except Exception as e:
                print(f"Error processing image {image_filename}: {e}")

if __name__ == "__main__":
    # Obtém o diretório de trabalho atual
    script_directory = os.getcwd()
    main(script_directory)

In [ ]:
# Classificação Torch e gerar CSV
import os
from transformers import pipeline
import csv

# Função para classificar uma imagem e retornar os resultados
def classify_image(image_path):
    results = classifier(image_path)
    return results

# Função para salvar os resultados em um arquivo CSV
def save_to_csv(results, csv_filename):
    with open(csv_filename, 'w', newline='') as csvfile:
        fieldnames = ['Image', 'Class', 'Confidence']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for result in results:
            image_name = os.path.basename(result['image'])
            class_label = result['label']
            confidence_percentage = result['score'] * 100
            writer.writerow({'Image': image_name, 'Class': class_label, 'Confidence': f'{confidence_percentage:.2f}%'})

# Carregar o pipeline de classificação de imagem
classifier = pipeline('image-classification', model='ramsformers/shoes-brand')

# Diretório onde estão as imagens .jpg
script_directory = os.getcwd()

# Lista todos os arquivos .jpg no diretório
jpg_files = [f for f in os.listdir(script_directory) if f.lower().endswith('.jpg')]

if not jpg_files:
    print("No .jpg files found in the script directory.")
else:
    # Armazenar resultados das classificações
    all_results = []

    for image_filename in jpg_files:
        image_path = os.path.join(script_directory, image_filename)

        try:
            results = classify_image(image_path)

            # Adiciona resultados ao conjunto global
            for result in results:
                result['image'] = image_path
            all_results.extend(results)

        except Exception as e:
            print(f"Error processing image {image_filename}: {e}")

    # Salvar os resultados em um arquivo CSV
    csv_filename = 'classification_results.csv'
    save_to_csv(all_results, csv_filename)
    print(f"Results saved to {csv_filename}"),

In [ ]:
# Classificação YoLO e gerar CSV
import os
from ultralyticsplus import YOLO, postprocess_classify_output
import csv

# Caminho para o diretório contendo as imagens
diretorio_imagens = os.getcwd()

# Carrega o modelo
modelo = YOLO('keremberke/yolov8m-shoe-classification')

# Define os parâmetros do modelo
modelo.overrides['conf'] = 0.25  # Limiar de confiança do modelo

# Lista para armazenar os resultados da classificação
resultados_classificacao = []

# Loop através de todas as imagens no diretório
for imagem_nome in os.listdir(diretorio_imagens):
    if imagem_nome.endswith('.jpg'):
        # Caminho completo da imagem
        caminho_imagem = os.path.join(diretorio_imagens, imagem_nome)

        # Realiza a inferência
        resultados = modelo.predict(caminho_imagem)

        # Obtém os resultados processados
        resultado_processado = postprocess_classify_output(modelo, result=resultados[0])

        # Obtém a classe com maior confiança
        classe_predita = max(resultado_processado, key=resultado_processado.get)

        # Obtém a confiança na classificação
        confianca = resultado_processado[classe_predita]

        # Armazena os resultados na lista
        resultados_classificacao.append({
            'Image': imagem_nome,
            'Class': classe_predita,
            'Confidence': f'{confianca:.2f}%'
        })
      
# Caminho para o arquivo CSV de resultados
caminho_arquivo_csv = 'classification_results.csv'

# Escreve os resultados no arquivo CSV
with open(caminho_arquivo_csv, 'w', newline='') as arquivo_csv:
    campos = ['Image', 'Class', 'Confidence']
    escritor_csv = csv.DictWriter(arquivo_csv, fieldnames=campos)

    # Escreve o cabeçalho
    escritor_csv.writeheader()

    # Escreve os dados
    for resultado in resultados_classificacao:
        escritor_csv.writerow(resultado)

print(f"Resultados da classificação foram salvos em: {caminho_arquivo_csv}")


In [ ]:
# Limpar CSV
import pandas as pd

# Lê o arquivo CSV de resultados
df = pd.read_csv('classification_results.csv')

# Converte a coluna 'Confidence' para tipo numérico
df['Confidence'] = df['Confidence'].str.rstrip('%').astype('float')

# Encontrar o índice do máximo para cada imagem
max_indexes = df.groupby('Image')['Confidence'].idxmax()

# Cria um novo DataFrame apenas com os registros correspondentes ao índice máximo
result_df = df.loc[max_indexes]

# Ordena o DataFrame pelo nome do arquivo em ordem alfabética
result_df = result_df.sort_values(by='Image')

# Salva o novo DataFrame em um arquivo CSV
result_df.to_csv('highest_confidence_per_image.csv', index=False)

In [ ]:
# Gerar Gráfico
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Ler o arquivo CSV para um DataFrame do Pandas
data = pd.read_csv('highest_confidence_per_image.csv')

# Filtrar as linhas que não têm o formato esperado
data = data[data['Image'].str.match(r'resized_\d+_\d+x\d+_.+\.jpg')]

# Extrair a resolução, valores de Confiança e Classes
def extract_resolution(image):
    parts = image.split('_')
    for part in parts:
        if 'x' in part:
            return tuple(map(int, part.split('x')))
    return (0, 0)

data['Resolution'] = data['Image'].apply(lambda x: extract_resolution(x)[0])  # Adiciona a coluna 'Resolution'

# Ordenar o DataFrame pela Largura (width) e Confiança em ordem decrescente
sorted_data = data.sort_values(by=['Resolution', 'Confidence', 'Class'], ascending=[False, False, True])

resolutions = sorted_data['Image'].apply(extract_resolution)
confidences = sorted_data['Confidence']
image_classes = sorted_data['Class']

# Criar um gráfico de barras
plt.figure(figsize=(12, 6))
bars = plt.bar(range(len(confidences)), confidences)

# Filtrar resoluções vazias antes de definir os ticks no eixo x
non_empty_resolutions = [f"{width}x{height}" for width, height in resolutions if width != 0 and height != 0]

if non_empty_resolutions:
    plt.xticks(range(len(non_empty_resolutions)), non_empty_resolutions, rotation=45, ha='right')

# Adicionar valores de confiança e classes em cima de cada barra
# for i, (bar, confidence, image_class) in enumerate(zip(bars, confidences, image_classes)):
#    width, height = resolutions.iloc[i]
#    plt.text(bar.get_x() + bar.get_width() / 2 - 0.15, bar.get_height() + 0.01,
#             f'{confidence}%', ha='center')  # Formatando para exibir duas casas decimais

# Criar um mapeamento de cores único para cada classe
class_colors = {cls: plt.cm.get_cmap('tab10')(i % 10) for i, cls in enumerate(set(image_classes))}

# Adicionar legenda com cor diferente para cada classe
legend_handles = [plt.Line2D([0], [0], color=class_colors[cls], label=cls) for cls in set(image_classes)]
plt.legend(handles=legend_handles)

# Atribuir cores às barras com base nas classes
bar_colors = [class_colors[cls] for cls in image_classes]
for bar, color in zip(bars, bar_colors):
    bar.set_color(color)

plt.xlabel('Resolução da Imagem')
plt.ylabel('Acertos (%)')
plt.title('Acertos por Resolução')

plt.tight_layout()

# Mostrar o gráfico
plt.show()